In [ ]:
#=================================================================================================================
# Movie creation codes
#=================================================================================================================

#-----------------------------------------------------------------------------------------------------------------   
def add_intro_credits(config, video_title):
    """
    Adds intro and credits
    """
    
    ## Intro title:
    print('\nGenerating intro...')

    n_frames_intro = np.int(config.t_intro*config.fps)
    f_on = np.int(config.t_intro_off1*config.fps)
    f_off = np.int((config.t_intro_off1 + config.t_intro_on)*config.fps)
    onoff_vector = np.full(n_frames_intro, False, dtype=bool)
    onoff_vector[f_on:f_off] = True
    
    for i, f_num in enumerate(range(-n_frames_intro, 0, 1)):

        frame = Frame(config, f_num)

        fig = plt.figure(figsize=(frame.figsizex, frame.figsizey), dpi=config.my_dpi)
        ax = fig.add_axes([0.,0.,1.,1.])

        ax.set_facecolor("black")
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        
        if onoff_vector[i]:
            ax.text(0.5, 0.5, video_title,
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes,
            color='white', fontsize=30)
            
        fig.savefig(frame.png_file)
        fig.clf()
        plt.close()  
        
        
    ## End credits:
    print('\nGenerating credits...')
        
    n_frames_end = np.int(config.t_credits*config.fps)
    f_on = np.int(config.t_credits_off1*config.fps)
    f_off = np.int((config.t_credits_off1 + config.t_credits_on)*config.fps)
    onoff_vector = np.full(n_frames_end, False, dtype=bool)
    onoff_vector[f_on:f_off] = True

    for i, f_num in enumerate(range(config.n_frames, config.n_frames + n_frames_end, 1)):
        
        frame = Frame(config, f_num)

        fig = plt.figure(figsize=(frame.figsizex, frame.figsizey), dpi=config.my_dpi)
        ax = fig.add_axes([0.,0.,1.,1.])

        ax.set_facecolor("black")
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        
        credits_version = 2

        if onoff_vector[i] and credits_version == 1:
            
            separation = 0.075

            ax.text(0.5, 0.5+separation/2, 'Animation and sound: Pablo Jimeno',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes,
            color='white', fontsize=20)

            ax.text(0.5, 0.5-separation/2, 'Created with data from: Wide-field Infrared Survey Explorer (WISE)',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes,
            color='white', fontsize=20)
            
            ax.text(0.5, 0.5-2*separation, '2018',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes,
            color='white', fontsize=20)
            
            
        if onoff_vector[i] and credits_version == 2:
            
            ax.text(0.5, 0.5+0.15, 'Animation and sound / Animación y sonido:',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes, color='white', fontsize=20)

            ax.text(0.5, 0.5+0.10, 'Pablo Jimeno',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes, color='white', fontsize=20)

            ax.text(0.5, 0.5, 'Generated with data from / Generado con datos de:',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes, color='white', fontsize=20)

            ax.text(0.5, 0.5-0.06, 'Wide-field Infrared Survey Explorer (WISE)',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes, color='white', fontsize=20)

            ax.text(0.5, 0.5-0.1, '[https://wise.ssl.berkeley.edu]',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes, color='white', fontsize=20)

            ax.text(0.5, 0.5-0.2, '2018',
            horizontalalignment='center',verticalalignment='center',transform=ax.transAxes,
            color='white', fontsize=20)

        fig.savefig(frame.png_file)
        fig.clf()
        plt.close()  
        
            
#-----------------------------------------------------------------------------------------------------------------   
def create_movie(config, audio_file, singularity_run, v_start=0, v_end=-1, intro_credits=True, renew_credits=True):
    """
    Creates the video from the frames generated and adds the sound
    """

    video_title = 'Singularity #%i' % singularity_run
    
    if intro_credits and v_start == 0 and v_end == -1 and renew_credits:
        add_intro_credits(config, video_title)
        
    if v_start == 0 and intro_credits:
        f_start = -np.int(config.t_intro*config.fps)
    else:
        f_start = np.int(np.floor(v_start/config.t_per_frame))
        
    if v_end == -1 and intro_credits:
        f_end = np.int(config.n_frames + config.t_credits*config.fps)
    else:
        f_end = np.int(np.ceil(v_end/config.t_per_frame))
        
    n_frames = f_end - f_start
    
    frames_dir = '%s/frames_final' % vid_dir
    
    movie_format='mp4'
        
    
    video_file_tag = 'singularity_#%i' % singularity_run
    
    mp3_file   = '%s/AUD/video_sound.mp3' % vid_dir
    video_file = '%s/VID/video_%s.mp4' % (vid_dir, video_file_tag)
    movie_file = '%s/movie_%s.%s' % (project_dir, video_file_tag, movie_format)

    for temp_file in [mp3_file, video_file, movie_file]:
        if os.path.exists(temp_file):
            os.remove(temp_file)
    
    print('\nGenerating .mp3 file from .wav...\n')
    !ffmpeg -i {audio_file} -vn -ar 44100 -ac 2 -ab 192k -f mp3 {mp3_file}
    print('\n','='*80, '\n')
    
    print('\nGenerating video from frames...\n')
    xsize = config.video_pixx
    ysize = np.int(1.*config.video_pixx/config.video_ratio)
    !ffmpeg -start_number {f_start} -r {config.fps} -f image2 -s {xsize}x{ysize} -i {frames_dir}/frame_%04d.png -vframes {n_frames} -vcodec libx264 -crf {config.crf_quality} -pix_fmt yuv420p {video_file}
    print('\n','='*80, '\n')
    
    print('\nAdding audio to video...\n')
    !ffmpeg -i {video_file} -i {mp3_file} -codec copy -shortest {movie_file}
    print('\n','='*80, '\n')

    assert os.path.exists(movie_file)
    print('\nMovie saved in: %s\n' % movie_file)
    print('\n','='*80, '\n')